In [44]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [45]:
import os

In [46]:
!pip install keras-unet

In [47]:
import keras
from keras_unet.models import custom_unet

In [48]:
DATAPATH = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/classified_images/npy_files'
images_file = os.path.join(DATAPATH, 'fly-jan-17-images.npy')
labels_file = os.path.join(DATAPATH, 'fly-jan-17-segmented.npy')

In [49]:
images = np.load(images_file)
labels = np.load(labels_file)

In [50]:
images.shape

(165, 500, 1500)

In [51]:
# full dataset does not have the last channel
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)

In [52]:
images.shape

(165, 500, 1500, 1)

In [53]:
p = np.random.permutation(len(images))

In [54]:
images = images[p]
labels = labels[p]

In [55]:
labels = labels.astype(np.float64)

In [56]:
images = images.astype(np.float64)

for i in range(images.shape[0]):
    images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually

In [57]:
images.shape

(165, 500, 1500, 1)

In [58]:
X_train = images[0:100]
y_train = labels[0:100]
X_val = images[100:150]
y_val = labels[100:150]
X_test = images[150:]
y_test = labels[150:]

In [61]:
model = custom_unet(
    input_shape=(512, 512, 1),
    use_batch_norm=False,
    num_classes=1,
    filters=32,
    dropout=0.5,
    output_activation='sigmoid')

In [62]:
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [63]:
opt = keras.optimizer_v1.Adam(lr=0.01)

In [64]:
model.compile(optimizer = 'Adam',    
              loss='binary_crossentropy', 
              metrics=[iou, iou_thresholded])

In [65]:
history = model.fit(X_train, 
                    y_train, 
                    32, 
                    epochs=100,
                    validation_data=(X_val, y_val))

Epoch 1/100


ValueError: in user code:

    /home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **k